In [2]:
import numpy as np

In [128]:
X = []
Y = []
with open('test.npy', 'rb') as f:
    data = np.load(f,allow_pickle=True)
    ievt = 0
    print('number of images in event', len(data[ievt]))
    ## Here we loop over all "images", which are created by integrating HCAL sampling layers defing 3 sectors defined 
    ## by two z position that define boundary. Note for all images ECAL is the same (no longitudinal separation in ECAL)
    for im in range(len(data[ievt])):
        event = np.c_[data[ievt][im]['HCAL1_x'][0],data[ievt][im]['HCAL1_y'][0],data[ievt][im]['HCAL1_E'][0]]
        depths = np.array([2*np.ones(len(data[ievt][im]['HCAL1_x'][0]))]).flatten()
        if (len(data[ievt][im]['HCAL2_x'])>0):
            event = np.concatenate([event,np.c_[data[ievt][im]['HCAL2_x'][0],data[ievt][im]['HCAL2_y'][0],data[ievt][im]['HCAL2_E'][0]]])
            depths = np.concatenate([depths,np.array([3*np.ones(len(data[ievt][im]['HCAL2_x'][0]))]).flatten()])
        if (len(data[ievt][im]['HCAL3_x'])>0):
            event = np.concatenate([event,np.c_[data[ievt][im]['HCAL3_x'][0],data[ievt][im]['HCAL3_y'][0],data[ievt][im]['HCAL3_E'][0]]])
            depths = np.concatenate([depths,np.array([4*np.ones(len(data[ievt][im]['HCAL3_x'][0]))]).flatten()])
        event = np.concatenate([event,np.c_[data[ievt][im]['ECAL_x'],data[ievt][im]['ECAL_y'],data[ievt][im]['ECAL_E']]])
        depths = np.concatenate([depths,np.array([1*np.ones(len(data[ievt][im]['ECAL_x']))]).flatten()])
        event = np.insert(event, 3, depths,axis=1)
        event = np.insert(event, 4, data[ievt][im]['boundary'][0]*np.ones(len(event)),axis=1) #better to make these global features, but I did not want to download the latest energyflow package
        event = np.insert(event, 5, data[ievt][im]['boundary'][1]*np.ones(len(event)),axis=1)
        X += [event]
        trueenergy = 1. #Miguel, please add this!
        Y += [trueenergy]
X = np.array(X)
Y = np.array(Y)

number of images in event 400


In [130]:
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split
#see https://energyflow.network/examples/

In [121]:
for x in X:
    xy_avg = np.average(x[:,0:2], axis=0)
    x[:,0:2] -= xy_avg
    x[:,2] /= 100. #could make this smarter
    x[:,4:6] /= 100.

In [134]:
(X_train, X_val, X_test,
 Y_train, Y_val, Y_test) = data_split(X, Y, val=0.2, test=0.3)

In [ ]:
#Probably want to standardize the input and output energies, or at least put them in units where the mean is O(1)

In [138]:
Phi_sizes, F_sizes = (100, 100, 128), (100, 100, 100)
pfn = PFN(input_dim=X.shape[-1], Phi_sizes=Phi_sizes, F_sizes=F_sizes, loss="mse")

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, 6)]    0                                            
__________________________________________________________________________________________________
tdist_0 (TimeDistributed)       (None, None, 100)    700         input[0][0]                      
__________________________________________________________________________________________________
activation_7 (Activation)       (None, None, 100)    0           tdist_0[0][0]                    
__________________________________________________________________________________________________
tdist_1 (TimeDistributed)       (None, None, 100)    10100       activation_7[0][0]               
____________________________________________________________________________________________

In [139]:
pfn.fit(X_train, Y_train,
        epochs=100,
        batch_size=100,
        validation_data=(X_val, Y_val),
        verbose=1)

Epoch 1/100
2/2 [==============================] - 1s 619ms/step - loss: 0.5000 - acc: 0.0000e+00 - val_loss: 0.5000 - val_acc: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s 34ms/step - loss: 0.5000 - acc: 0.0000e+00 - val_loss: 0.5000 - val_acc: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s 45ms/step - loss: 0.5000 - acc: 0.0000e+00 - val_loss: 0.5000 - val_acc: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 0s 41ms/step - loss: 0.5000 - acc: 0.0000e+00 - val_loss: 0.5000 - val_acc: 0.0000e+00
Epoch 5/100
2/2 [==============================] - 0s 39ms/step - loss: 0.5000 - acc: 0.0000e+00 - val_loss: 0.5000 - val_acc: 0.0000e+00
Epoch 6/100
2/2 [==============================] - 0s 40ms/step - loss: 0.5000 - acc: 0.0000e+00 - val_loss: 0.5000 - val_acc: 0.0000e+00
Epoch 7/100
2/2 [==============================] - 0s 42ms/step - loss: 0.5000 - acc: 0.0000e+00 - val_loss: 0.5000 - val_acc: 0.0000e+00
Epoch 8/100
2/2 [================

Epoch 60/100
2/2 [==============================] - 0s 37ms/step - loss: 0.5000 - acc: 0.0000e+00 - val_loss: 0.5000 - val_acc: 0.0000e+00
Epoch 61/100
2/2 [==============================] - 0s 38ms/step - loss: 0.5000 - acc: 0.0000e+00 - val_loss: 0.5000 - val_acc: 0.0000e+00
Epoch 62/100
2/2 [==============================] - 0s 38ms/step - loss: 0.5000 - acc: 0.0000e+00 - val_loss: 0.5000 - val_acc: 0.0000e+00
Epoch 63/100
2/2 [==============================] - 0s 38ms/step - loss: 0.5000 - acc: 0.0000e+00 - val_loss: 0.5000 - val_acc: 0.0000e+00
Epoch 64/100
2/2 [==============================] - 0s 39ms/step - loss: 0.5000 - acc: 0.0000e+00 - val_loss: 0.5000 - val_acc: 0.0000e+00
Epoch 65/100
2/2 [==============================] - 0s 37ms/step - loss: 0.5000 - acc: 0.0000e+00 - val_loss: 0.5000 - val_acc: 0.0000e+00
Epoch 66/100
2/2 [==============================] - 0s 38ms/step - loss: 0.5000 - acc: 0.0000e+00 - val_loss: 0.5000 - val_acc: 0.0000e+00
Epoch 67/100
2/2 [=========

In [140]:
mypreds = pfn.predict(X_test,batch_size=100)

In [141]:
plt.scatter(Y_test,mypreds)

NameError: name 'plt' is not defined